## Additive manufacturing audio signal processing - V2
- Author： Chen Lequn
- Oct 2021
- A comprehensive analysis on additive manufacturing acoustic signal.
- System requirement: Python3

In [ ]:
## Required python libraries
import pandas as pd
import numpy as np
import scipy as sp
import os
import math

# to make this notebook's output stable across runs
np.random.seed(42)

pd.plotting.register_matplotlib_converters()


import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [ ]:
## Audio signal processing libraries
import librosa
import librosa.display
import IPython.display as ipd

In [ ]:
# Pytorch libraryes and torchaudio - for GPU accelerated feature extraction
import torch
import torchaudio
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchsummary import summary

In [ ]:
# Where to save the figures, and dataset locations
PROJECT_ROOT_DIR = "."
Audio_PATH = os.path.join(PROJECT_ROOT_DIR, "AM_audio_dataset")
ABB_LAAM_printing_path = os.path.join(Audio_PATH, "abb_laam")
kuka_wire_LAAM_printing_path = os.path.join(Audio_PATH, "kuka_wire_laam")
gas_powder_path = os.path.join(Audio_PATH, "gas_powder")
cnc_prinitng = os.path.join(Audio_PATH, "cnc_printing")
ambient_path = os.path.join(Audio_PATH, "ambient")

IMAGE_PATH = os.path.join(PROJECT_ROOT_DIR, "signal_processing_images")
os.makedirs(Audio_PATH, exist_ok=True)
os.makedirs(IMAGE_PATH, exist_ok=True)


## function for automatically save the diagram/graph into the folder 
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Load Sound

In [ ]:
# load sounds
LAAM1, sampling_rate = librosa.load(os.path.join(ABB_LAAM_printing_path, "printing_normal_1.wav"))
LAAM2, _ = librosa.load(os.path.join(ABB_LAAM_printing_path, "printing_normal_3.wav"))
LAAM3, _ = librosa.load(os.path.join(ABB_LAAM_printing_path, "printing_normal_5.wav"))
LAAM4, _  = librosa.load(os.path.join(ABB_LAAM_printing_path, "printing4.wav"))

cnc1, _ = librosa.load(os.path.join(cnc_prinitng, "3.wav"))
cnc1, _ = librosa.load(os.path.join(cnc_prinitng, "4.wav"))
cnc1, _ = librosa.load(os.path.join(cnc_prinitng, "5.wav"))


ambient1, _ = librosa.load(os.path.join(ambient_path, "aimbient914.wav"))
ambient2, _ = librosa.load(os.path.join(ambient_path, "aimbient5.wav"))


gas1, _ = librosa.load(os.path.join(gas_powder_path, "gas2.wav"))
gas2, _ = librosa.load(os.path.join(gas_powder_path, "printing_gas_powder.wav"))

Kuka1, _ = librosa.load(os.path.join(kuka_wire_LAAM_printing_path, "9.wav"))
Kuka2, _ = librosa.load(os.path.join(kuka_wire_LAAM_printing_path, "10.wav"))
Kuka3, _ = librosa.load(os.path.join(kuka_wire_LAAM_printing_path, "8.wav"))